## DataSet

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


df = pd.read_csv('normalized.csv')
X = df.iloc[:,:64].to_numpy()
y= df.iloc[:,64].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.astype(np.float32)
X_test.astype(np.float32)
y_train.astype(np.float32)
y_test.astype(np.float32)

batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
y_train = y_train - 1
y_test = y_test - 1


train_lables = to_categorical(y_train)
eval_labels = to_categorical(y_test)



## Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization


# DEfine the model
model = Sequential()
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(Dense(5))
model.add(Activation('softmax'))


# Iniyalize
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, train_lables, epochs=100, batch_size=128)

Epoch 1/100
53/53 [==============================] - 2s 4ms/step - loss: 1.5198 - accuracy: 0.4168
Epoch 2/100
53/53 [==============================] - 0s 4ms/step - loss: 1.1429 - accuracy: 0.5441
Epoch 3/100
53/53 [==============================] - 0s 4ms/step - loss: 0.9936 - accuracy: 0.6118
Epoch 4/100
53/53 [==============================] - 0s 4ms/step - loss: 0.8979 - accuracy: 0.6501
Epoch 5/100
53/53 [==============================] - 0s 4ms/step - loss: 0.8349 - accuracy: 0.6778
Epoch 6/100
53/53 [==============================] - 0s 4ms/step - loss: 0.7917 - accuracy: 0.6917
Epoch 7/100
53/53 [==============================] - 0s 5ms/step - loss: 0.7529 - accuracy: 0.7108
Epoch 8/100
53/53 [==============================] - 0s 6ms/step - loss: 0.7250 - accuracy: 0.7190
Epoch 9/100
53/53 [==============================] - 0s 5ms/step - loss: 0.7019 - accuracy: 0.7227
Epoch 10/100
53/53 [==============================] - 0s 9ms/step - loss: 0.6735 - accuracy: 0.7393
Epoch 11/

## Model Save
- original model
- lite model
- quantize model

In [ ]:
import tensorflow as tf
import pathlib

# filename
file_name = "normalize"

# transform TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Convert with Dynamic Range Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert with Floating-Point Fallback Quantization
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter.representative_dataset = representative_data_gen

# Convert using integer-only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# end of quantization
tflite_model_quant = converter.convert()

# save
tflite_models_dir = pathlib.Path("/content/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# save the original model
model.save('{}.h5'.format(file_name))

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"{}.tflite".format(file_name)
tflite_model_file.write_bytes(tflite_model)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"{}_quant.tflite".format(file_name)
tflite_model_quant_file.write_bytes(tflite_model_quant)


## Header conversion

In [ ]:
!xxd -i /content/my_model.tflite > /content/model_quant.h